In [ ]:
# Možeš lokalno da ubaciš dataset (ako ga skineš kod sebe)
from google.colab import files
uploaded = files.upload()

Saving wikipedia-pageviews-20240101 to wikipedia-pageviews-20240101 (1)


In [ ]:
# Povezivanje da ti pronađe dataset preko drive-a
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import csv

filename = "/content/wikipedia-pageviews-20240101"

df = pd.read_csv(
    filename,
    sep=r"\s+",                 # bilo koji whitespace
    header=None,
    names=["project", "title", "views", "bytes"],
    engine="python",
    quoting=csv.QUOTE_NONE,     # ignoriši navodnike
    encoding="utf-8",
    on_bad_lines="skip"         # ako je neki red baš pokvaren, preskoči ga
)

print(df.head())
print(df.shape)


  project                         title  views  bytes
0      ""  Category:Account_creators/en      1      0
1      ""             Category:Archives      1      0
2      ""          Category:Archives/ar      1      0
3      ""          Category:Archives/de      1      0
4      ""          Category:Archives/es      1      0
(5197721, 4)


In [ ]:
df.shape

(5197721, 4)

In [ ]:
print("Ukupan broj redova:", len(df))

Ukupan broj redova: 5197721


In [ ]:
df['project'].unique()

array(['""', 'aa', 'aa.b', ..., 'zu.d', 'zu.m', 'zu.m.d'], dtype=object)

In [ ]:
df['project'].value_counts()

,count
project,
en.m,1235194
en,795166
es.m,235523
fr.m,209325
de.m,198448
...,...
ho.m,1
zh-min-nan.m.s,1
zu.d,1


In [ ]:
df['project'].value_counts().head(20)

,count
project,
en.m,1235194
en,795166
es.m,235523
fr.m,209325
de.m,198448
ru.m,192222
ja.m,178769
it.m,147711
pt.m,100086


In [ ]:
mask_en = df['project'].map(lambda x: x[:2] == 'en')
df.loc[mask_en, 'project'].value_counts()

,count
project,
en.m,1235194
en,795166
en.m.d,37675
en.d,28161
en.s,1790
en.voy,1365
en.m.voy,1318
en.m.q,1271
en.q,1231


In [ ]:
df[df['project'].isin(["en", "en.m"])].shape

(2030360, 4)

In [ ]:
#Spajanje u jedan dataset podataka proj en i en.m
valid_projects = ["en", "en.m"]
df_en = df[df['project'].isin(valid_projects)]

In [ ]:
# Spajanje u jedan red dupliciranih istih pod koji su se tražili i preko en i en_m
df_en_merged = (
    df_en
    .groupby("title", as_index=False)
    .agg({
        "views": "sum",
        "bytes": "sum"   # možeš i izbaciti ako ti bytes ne trebaju
    })
)

In [ ]:
df_en_merged["project"] = "en_merged"

In [ ]:
df_en_merged = df_en_merged[["project", "title", "views", "bytes"]]

In [ ]:
print("Broj redova prije spajanja:", len(df_en))
print("Broj redova poslije spajanja:", len(df_en_merged))

Broj redova prije spajanja: 2030360
Broj redova poslije spajanja: 1563030


In [ ]:
# pravljenje liste sa svim podacima naslova jedan iza drugog onoliko puta koliko su se tražili
import numpy as np

access_sequence = np.repeat(
    df_en_merged["title"].values,
    df_en_merged["views"].values
)

In [ ]:
len(access_sequence)

11968317

In [ ]:
# Exportujemo niz
with open("access_sequence.txt", "w") as f:
    for item in access_sequence:
        f.write(item + "\n")

In [ ]:
import os
os.listdir('/content')


['.config',
 'access_sequence.txt',
 '.ipynb_checkpoints',
 'wikipedia-pageviews-20240101',
 'sample_data']

In [ ]:
# Pravimo novi niz - samo one stranice koje imaju bar 50 pregleda
df_small = df_en_merged[df_en_merged["views"] >= 50].copy()

import numpy as np
access_sequence_small = np.repeat(
    df_small["title"].values,
    df_small["views"].values
)

len(access_sequence_small)


6139104

In [ ]:
# Exportujemo niz
with open("access_sequence_small.txt", "w") as f:
    for item in access_sequence_small:
        f.write(item + "\n")

In [ ]:
import os
os.listdir('/content')


['.config',
 'access_sequence.txt',
 '.ipynb_checkpoints',
 'wikipedia-pageviews-20240101',
 'access_sequence_small.txt',
 'sample_data']

In [ ]:
seq_test = access_sequence_small[:200_000]
len(seq_test)

200000

In [ ]:
# Implementacija Move-To-Front u Pythonu
import numpy as np

def move_to_front(access_sequence):
    """
    access_sequence: niz pristupa (npr. access_sequence_small)

    Vraća:
      - final_list: stanje liste nakon svih pristupa
      - total_cost: ukupan trošak pristupa
      - access_costs: lista troškova po pristupu
    """
    # početno stanje liste: elementi redom kako se prvi put pojave
    current_list = []
    seen = set()
    for item in access_sequence:
        if item not in seen:
            seen.add(item)
            current_list.append(item)

    total_cost = 0
    access_costs = []

    for key in access_sequence:
        # pronađi poziciju (linearna pretraga, kao u jednostavnoj listi)
        idx = current_list.index(key)      # O(n)
        cost = idx + 1                     # pozicija + 1
        total_cost += cost
        access_costs.append(cost)

        # pomjeri element na početak (front)
        if idx != 0:
            current_list.pop(idx)
            current_list.insert(0, key)

    return current_list, total_cost, access_costs


In [ ]:
# poziv i rezultati

final_list_mtf, total_cost_mtf, access_costs_mtf = move_to_front(seq_test)

print("Broj pristupa:", len(seq_test))
print("Ukupan trošak (MTF):", total_cost_mtf)
print("Prosječni trošak po pristupu:", total_cost_mtf / len(seq_test))


Broj pristupa: 2000000
Ukupan trošak (MTF): 75198950
Prosječni trošak po pristupu: 37.599475


In [ ]:
# Implementacija Count u Pythonu

# Implementacija Count heuristike u Pythonu

def count_algorithm(access_sequence):

    # početno stanje liste: elementi redom kako se prvi put pojave
    current_list = []
    counts = []          # paralelni niz koji čuva count za svaki element

    total_cost = 0
    access_costs = []

    for key in access_sequence:
        # provjera da li je element već u listi
        if key in current_list:
            idx = current_list.index(key)
        else:
            # novi element – dodaj ga na kraj liste sa početnim brojem pristupa 0
            current_list.append(key)
            counts.append(0)
            idx = len(current_list) - 1

        # trošak pristupa je pozicija + 1
        cost = idx + 1
        total_cost += cost
        access_costs.append(cost)

        # uvećaj broj pristupa za ovaj element
        counts[idx] += 1

        # "bubble up": pomjeraj element prema početku liste
        # sve dok ima veći count od elemenata ispred sebe
        while idx > 0 and counts[idx] > counts[idx - 1]:
            # zamijeni mjesta u oba niza (i ključ i njegov count)
            counts[idx], counts[idx - 1] = counts[idx - 1], counts[idx]
            current_list[idx], current_list[idx - 1] = current_list[idx - 1], current_list[idx]
            idx -= 1

    return current_list, counts, total_cost, access_costs


In [ ]:
final_list_count, counts_count, total_cost_count, access_costs_count = count_algorithm(seq_test)

print("=== REZULTATI – COUNT HEURISTIKA ===")
print("Broj pristupa:", len(seq_test))
print("Ukupan trošak (COUNT):", total_cost_count)
print("Prosječni trošak po pristupu (COUNT):", total_cost_count / len(seq_test))


=== REZULTATI – COUNT HEURISTIKA ===
Broj pristupa: 200000
Ukupan trošak (COUNT): 54481018
Prosječni trošak po pristupu (COUNT): 272.40509


In [ ]:
print("=== POREĐENJE MTF vs COUNT ===")
print("Ukupan trošak (MTF):   ", total_cost_mtf)
print("Ukupan trošak (COUNT): ", total_cost_count)
print("Prosječni trošak (MTF):   ", total_cost_mtf / len(seq_test))
print("Prosječni trošak (COUNT): ", total_cost_count / len(seq_test))